In [1]:
%pip install -q gradio

In [2]:
%pip install -q langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.0 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.0.0 which is incompatible.


In [3]:
import gradio as gr
from langchain_community.document_loaders import PyPDFLoader
from google import genai
from google.genai import types
import wave
import os
import getpass

In [4]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 6.4 MB/s eta 0:00:00


In [5]:
# Set up the wave file to save the output:
def wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
   with wave.open(filename, "wb") as wf:
      wf.setnchannels(channels)
      wf.setsampwidth(sample_width)
      wf.setframerate(rate)
      wf.writeframes(pcm)

In [6]:
def process_pdf_to_audio(pdf_file, api_key):
    # Initialize the client with the API key provided in the UI
    client = genai.Client(api_key=api_key)

    # Load the PDF
    loader = PyPDFLoader(pdf_file)
    docs = loader.load()

    # Combine all pages into a single text
    input_text = "\n".join([doc.page_content for doc in docs])

    prompt = f"""
    Your task is to extract relevant information.

    From the text below, delimited by triple quotes \
    extract the information relevant
    Also give output in paragraph format without any * : ```{input_text}```
    """

    # Extract information using Gemini
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        config=types.GenerateContentConfig(
            system_instruction="you are a helpful assisant"),
        contents=prompt
    )

    # Get the extracted text
    extracted_text = response.candidates[0].content.parts[0].text


    # Use gemini-2.5-flash-preview-tts to convert the generated transcript to audio
    response = client.models.generate_content(
      model="gemini-2.5-flash-preview-tts",
      contents=extracted_text,
      config=types.GenerateContentConfig(
          response_modalities=["AUDIO"],
          speech_config=types.SpeechConfig(
            voice_config=types.VoiceConfig(
                prebuilt_voice_config=types.PrebuiltVoiceConfig(
                  voice_name='Gacrux',
                )
            )
          ),
      )
    )

    audio_data = response.candidates[0].content.parts[0].inline_data.data
    output_filename = "output.wav"
    wave_file(output_filename, audio_data)

    return output_filename

In [7]:
# Create the Gradio interface
iface = gr.Interface(
    fn=process_pdf_to_audio,
    inputs=[gr.File(label="Upload PDF File"), gr.Textbox(label="Enter your Google API Key")],
    outputs=gr.Audio(label="Generated Audio"),
    title="PDF to Audio Podcast",
    description="Upload a PDF file and enter your Google API Key to generate an audio podcast based on its extracted content."
)

In [11]:
# Launch the Gradio interface
iface.launch(inline=True)

In [12]:
iface.close()